In [ ]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

As the silver moon sparkled in the sky, a gentle unicorn tiptoed through the dream-dust meadow, leaving trails of magic that tucked every sleepy child into sweet, twinkling dreams.


In [ ]:
from google.colab import userdata
from agents import AsyncOpenAI, Agent, function_tool, Runner, OpenAIChatCompletionsModel, set_tracing_disabled
import json


API_KEY = userdata.get('GEMINI_API_KEY')
MODEL = "gemini-2.0-flash"

external_client = AsyncOpenAI(
    api_key=API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

@function_tool
def get_current_weather(location:str, unit:str="celsius")->str:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


agent2 = Agent(
    name="Piaic assistant",
    instructions="You are good at handling PIAIC related queries",
    model=OpenAIChatCompletionsModel(model=MODEL, openai_client=external_client),
    handoff_description="PIAIC expert"
  )
agent3 = agent2.clone(
    name="Weather assistant",
    instructions="You are good at handling weather related queries",
    tools=[get_current_weather],
    handoff_description="Weather expert"
  )

agent1 = Agent(
    name="Assistant",
    instructions="You are good at giving answers to the users queries. you can handoff tasks to the other agents",
    model=OpenAIChatCompletionsModel(model=MODEL, openai_client=external_client),
    handoffs=[agent2, agent3]
  )

set_tracing_disabled(disabled=True)


Assistant
How can I assist you today?



In [ ]:

result = Runner.run_sync(
    starting_agent=agent1,
    input="What are the courses offered by PIAIC")

print(result.last_agent.name)
print(result.final_output)

Piaic assistant
PIAIC (Presidential Initiative for Artificial Intelligence & Computing) offers a range of courses focused on in-demand technology skills. Here's a breakdown of the main programs:

*   **Artificial Intelligence (AI):** This is their flagship program, covering the fundamentals of AI, machine learning, deep learning, and related concepts.
*   **Cloud Native Computing:** This program focuses on modern application development techniques using cloud technologies like containers, microservices, and DevOps practices.
*   **Web 3.0 (Blockchain):** This course delves into blockchain technology, cryptocurrencies, decentralized applications (dApps), and related concepts.
*   **Internet of Things (IoT):** The IoT program covers the technologies and applications of interconnected devices, data analytics, and automation in various industries.

PIAIC also often introduces specialized workshops, bootcamps, and shorter courses on specific topics within these broader areas. For the most u

In [ ]:

result = Runner.run_sync(
    starting_agent=agent1,
    input="What is the weather in Tokyo in celcius")

print(result.last_agent.name)
print(result.final_output)

Weather assistant
The weather in Tokyo is 10 degrees Celsius.



In [ ]:
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

In [ ]:

result = Runner.run_sync(
    starting_agent=agent1,
    input="HI")

print(result.last_agent.name)
print(result.final_output)

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7838121082f0>


...

LLM resp:
{
  "content": "How can I help you today?\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "How can I help you today?\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Assistant
How can I help you today?



In [ ]:

result = Runner.run_sync(
    starting_agent=agent1,
    input="What are the courses offered by PIAIC")

print(result.last_agent.name)
print(result.final_output)

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x78381315fbf0>


...

LLM resp:
{
  "content": "PIAIC (Presidential Initiative for Artificial Intelligence & Computing) offers a range of courses focused on in-demand technologies. Here's a breakdown of the courses they typically offer:\n\n*   **Artificial Intelligence (AI):** This is a flagship program covering the fundamentals of AI, machine learning, deep learning, and related concepts.\n\n*   **Cloud Native Computing:** This course focuses on developing and deploying applications using cloud-native technologies like containers, Kubernetes, and microservices.\n\n*   **Web 3.0 and Metaverse:** A course covering blockchain technologies, decentralized applications (dApps), NFTs, and the Metaverse concepts.\n\n*   **Internet of Things (IoT):** This course explores the world of connected devices, sensors, and data analysis in IoT environments.\n\n**Important Notes:**\n\n*   **Course Availability:** The specific courses offered by PIAIC may vary from time to time. Check the PIAIC official website (www.piaic.or

DEBUG:openai.agents:LLM resp:
{
  "content": "PIAIC (Presidential Initiative for Artificial Intelligence & Computing) offers a range of courses focused on in-demand technologies. Here's a breakdown of the courses they typically offer:\n\n*   **Artificial Intelligence (AI):** This is a flagship program covering the fundamentals of AI, machine learning, deep learning, and related concepts.\n\n*   **Cloud Native Computing:** This course focuses on developing and deploying applications using cloud-native technologies like containers, Kubernetes, and microservices.\n\n*   **Web 3.0 and Metaverse:** A course covering blockchain technologies, decentralized applications (dApps), NFTs, and the Metaverse concepts.\n\n*   **Internet of Things (IoT):** This course explores the world of connected devices, sensors, and data analysis in IoT environments.\n\n**Important Notes:**\n\n*   **Course Availability:** The specific courses offered by PIAIC may vary from time to time. Check the PIAIC official w

Resetting current trace


DEBUG:openai.agents:Resetting current trace


Piaic assistant
PIAIC (Presidential Initiative for Artificial Intelligence & Computing) offers a range of courses focused on in-demand technologies. Here's a breakdown of the courses they typically offer:

*   **Artificial Intelligence (AI):** This is a flagship program covering the fundamentals of AI, machine learning, deep learning, and related concepts.

*   **Cloud Native Computing:** This course focuses on developing and deploying applications using cloud-native technologies like containers, Kubernetes, and microservices.

*   **Web 3.0 and Metaverse:** A course covering blockchain technologies, decentralized applications (dApps), NFTs, and the Metaverse concepts.

*   **Internet of Things (IoT):** This course explores the world of connected devices, sensors, and data analysis in IoT environments.

**Important Notes:**

*   **Course Availability:** The specific courses offered by PIAIC may vary from time to time. Check the PIAIC official website (www.piaic.org) for the most up-to-

In [ ]:

result = Runner.run_sync(
    starting_agent=agent1,
    input="What is the weather in Tokyo in celcius")

print(result.last_agent.name)
print(result.final_output)

Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7838121082f0>


...

LLM resp:
{
  "content": "The weather in Tokyo is 10 degrees Celsius.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "The weather in Tokyo is 10 degrees Celsius.\n",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Weather assistant
The weather in Tokyo is 10 degrees Celsius.

